In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


#### (Upload Data from CSV into a Snowflake Table)

In [ ]:
SELECT *
FROM MARKOV_MODEL_DB.CUSTOMER_INTERACTIONS.CUSTOMER_JOURNEY_DATA
LIMIT 5;

## Page Transitions Matrix

In [ ]:
CREATE OR REPLACE TABLE PAGE_TRANSITIONS AS
WITH ordered_events AS (
    -- Order all events within each session by timestamp
    SELECT 
        CUSTOMER_ID,
        SESSION_ID,
        EVENT_TIMESTAMP,
        PAGE_NAME AS CURRENT_PAGE,
        DEVICE_TYPE,
        HOUR(EVENT_TIMESTAMP) AS EVENT_HOUR,
        -- Time of day buckets: morning (6-11), afternoon (12-17), evening (18-23), night (0-5)
        CASE 
            WHEN HOUR(EVENT_TIMESTAMP) BETWEEN 6 AND 11 THEN 'morning'
            WHEN HOUR(EVENT_TIMESTAMP) BETWEEN 12 AND 17 THEN 'afternoon'
            WHEN HOUR(EVENT_TIMESTAMP) BETWEEN 18 AND 23 THEN 'evening'
            ELSE 'night'
        END AS TIME_OF_DAY,
        -- Get the next page in the session
        LEAD(PAGE_NAME) OVER (
            PARTITION BY SESSION_ID 
            ORDER BY EVENT_TIMESTAMP
        ) AS NEXT_PAGE,
        -- Flag if this is the last event in the session
        CASE 
            WHEN LEAD(PAGE_NAME) OVER (PARTITION BY SESSION_ID ORDER BY EVENT_TIMESTAMP) IS NULL 
            THEN TRUE 
            ELSE FALSE 
        END AS IS_LAST_EVENT,
        CONVERTED
    FROM MARKOV_MODEL_DB.CUSTOMER_INTERACTIONS.CUSTOMER_JOURNEY_DATA
),
transitions_with_exit AS (
    -- Add transitions to EXIT state
    SELECT 
        CUSTOMER_ID,
        SESSION_ID,
        EVENT_TIMESTAMP,
        CURRENT_PAGE,
        DEVICE_TYPE,
        TIME_OF_DAY,
        -- If it's the last event and didn't convert, next page is EXIT
        -- Otherwise use the actual next page
        CASE 
            WHEN IS_LAST_EVENT AND NOT CONVERTED THEN 'EXIT'
            WHEN IS_LAST_EVENT AND CONVERTED THEN NEXT_PAGE
            ELSE NEXT_PAGE
        END AS NEXT_PAGE,
        CONVERTED
    FROM ordered_events
    WHERE NEXT_PAGE IS NOT NULL OR IS_LAST_EVENT = TRUE
)
SELECT * FROM transitions_with_exit;

In [ ]:
SELECT *
FROM PAGE_TRANSITIONS
LIMIT 5;

In [ ]:
-- Calculate transition counts and probabilities

CREATE OR REPLACE TABLE TRANSITION_MATRIX AS
WITH transition_counts AS (
    -- Count transitions for each combination
    SELECT 
        CURRENT_PAGE,
        NEXT_PAGE,
        DEVICE_TYPE,
        TIME_OF_DAY,
        COUNT(*) AS TRANSITION_COUNT
    FROM PAGE_TRANSITIONS
    GROUP BY CURRENT_PAGE, NEXT_PAGE, DEVICE_TYPE, TIME_OF_DAY
),
total_from_page AS (
    -- Total transitions from each page (by device and time)
    SELECT 
        CURRENT_PAGE,
        DEVICE_TYPE,
        TIME_OF_DAY,
        SUM(TRANSITION_COUNT) AS TOTAL_TRANSITIONS
    FROM transition_counts
    GROUP BY CURRENT_PAGE, DEVICE_TYPE, TIME_OF_DAY
)
SELECT 
    tc.CURRENT_PAGE,
    tc.NEXT_PAGE,
    tc.DEVICE_TYPE,
    tc.TIME_OF_DAY,
    tc.TRANSITION_COUNT,
    tfp.TOTAL_TRANSITIONS,
    -- Calculate probability: P(next_page | current_page, device, time)
    ROUND(tc.TRANSITION_COUNT::FLOAT / tfp.TOTAL_TRANSITIONS, 4) AS TRANSITION_PROBABILITY
FROM transition_counts tc
JOIN total_from_page tfp
    ON tc.CURRENT_PAGE = tfp.CURRENT_PAGE
    AND tc.DEVICE_TYPE = tfp.DEVICE_TYPE
    AND tc.TIME_OF_DAY = tfp.TIME_OF_DAY
ORDER BY 
    tc.CURRENT_PAGE, 
    tc.DEVICE_TYPE,
    tc.TIME_OF_DAY,
    TRANSITION_PROBABILITY DESC;


In [ ]:
SELECT *
FROM TRANSITION_MATRIX;

### Verification Queries

In [ ]:
-- Check total transitions created
SELECT 
    COUNT(*) AS TOTAL_TRANSITION_RECORDS,
    SUM(TRANSITION_COUNT) AS TOTAL_TRANSITIONS
FROM TRANSITION_MATRIX;

In [ ]:
-- View sample transitions from 'cart' page on mobile during afternoon
SELECT 
    CURRENT_PAGE,
    NEXT_PAGE,
    DEVICE_TYPE,
    TIME_OF_DAY,
    TRANSITION_COUNT,
    TRANSITION_PROBABILITY,
    ROUND(TRANSITION_PROBABILITY * 100, 2) AS PROBABILITY_PCT
FROM TRANSITION_MATRIX
WHERE CURRENT_PAGE = 'cart'
    AND DEVICE_TYPE = 'mobile'
    AND TIME_OF_DAY = 'afternoon'
ORDER BY TRANSITION_PROBABILITY DESC;

In [ ]:
-- See which pages have highest exit rates
SELECT 
    CURRENT_PAGE,
    DEVICE_TYPE,
    TIME_OF_DAY,
    TRANSITION_PROBABILITY AS EXIT_RATE,
    ROUND(TRANSITION_PROBABILITY * 100, 2) AS EXIT_RATE_PCT
FROM TRANSITION_MATRIX
WHERE NEXT_PAGE = 'EXIT'
ORDER BY EXIT_RATE DESC
LIMIT 20;

In [ ]:
-- Compare conversion paths: desktop vs mobile in afternoon
SELECT 
    'desktop' AS device,
    NEXT_PAGE,
    TRANSITION_PROBABILITY
FROM TRANSITION_MATRIX
WHERE CURRENT_PAGE = 'cart' 
    AND DEVICE_TYPE = 'desktop'
    AND TIME_OF_DAY = 'afternoon'
UNION ALL
SELECT 
    'mobile' AS device,
    NEXT_PAGE,
    TRANSITION_PROBABILITY
FROM TRANSITION_MATRIX
WHERE CURRENT_PAGE = 'cart' 
    AND DEVICE_TYPE = 'mobile'
    AND TIME_OF_DAY = 'afternoon'
ORDER BY device, TRANSITION_PROBABILITY DESC;

### Summary Statistics

In [ ]:
SELECT 
    'Total unique transitions' AS metric,
    COUNT(*) AS value
FROM TRANSITION_MATRIX
UNION ALL
SELECT 
    'Unique current pages',
    COUNT(DISTINCT CURRENT_PAGE)
FROM TRANSITION_MATRIX
UNION ALL
SELECT 
    'Unique next pages',
    COUNT(DISTINCT NEXT_PAGE)
FROM TRANSITION_MATRIX
UNION ALL
SELECT 
    'Device types',
    COUNT(DISTINCT DEVICE_TYPE)
FROM TRANSITION_MATRIX
UNION ALL
SELECT 
    'Time of day segments',
    COUNT(DISTINCT TIME_OF_DAY)
FROM TRANSITION_MATRIX;

## Building the Python Function

In [ ]:
-- STEP 2: CREATE MARKOV CHAIN ANALYSIS PROCEDURE

CREATE OR REPLACE PROCEDURE MARKOV_CHAIN_ANALYSIS(
    MODE VARCHAR,
    CURRENT_PAGE VARCHAR,
    DEVICE_TYPE VARCHAR,
    TIME_OF_DAY VARCHAR,
    PARAMS VARCHAR
)
RETURNS VARIANT
LANGUAGE PYTHON
RUNTIME_VERSION = '3.10'
PACKAGES = ('snowflake-snowpark-python')
HANDLER = 'analyze'
AS
$$
import json
import random
from collections import defaultdict, Counter

def analyze(session, mode, current_page, device_type, time_of_day, params):
    """
    Main handler for Markov chain analysis
    
    Modes:
    - 'predict_next': Get most likely next pages with probabilities
    - 'conversion_probability': Calculate probability of reaching checkout
    - 'simulate_journey': Run Monte Carlo simulation of customer journeys
    - 'common_paths': Find most common paths between two pages
    """
    
    # Parse params if it's a string
    if isinstance(params, str):
        params = json.loads(params) if params else {}
    elif params is None:
        params = {}
    
    # Normalize inputs
    mode = mode.lower() if mode else 'predict_next'
    current_page = current_page.lower() if current_page else None
    device_type = device_type.lower() if device_type else 'mobile'
    time_of_day = time_of_day.lower() if time_of_day else 'afternoon'
    
    # Route to appropriate analysis function
    if mode == 'predict_next':
        return predict_next_page(session, current_page, device_type, time_of_day, params)
    elif mode == 'conversion_probability':
        return calculate_conversion_probability(session, current_page, device_type, time_of_day, params)
    elif mode == 'simulate_journey':
        return simulate_journey(session, current_page, device_type, time_of_day, params)
    elif mode == 'common_paths':
        return find_common_paths(session, device_type, time_of_day, params)
    else:
        return {"error": f"Unknown mode: {mode}. Valid modes: predict_next, conversion_probability, simulate_journey, common_paths"}


def get_transitions(session, from_page, device_type, time_of_day):
    """
    Get all transitions from a given page for specific device and time
    Returns dict: {next_page: probability}
    """
    query = f"""
        SELECT NEXT_PAGE, TRANSITION_PROBABILITY
        FROM TRANSITION_MATRIX
        WHERE CURRENT_PAGE = '{from_page}'
          AND DEVICE_TYPE = '{device_type}'
          AND TIME_OF_DAY = '{time_of_day}'
    """
    
    result = session.sql(query).collect()
    transitions = {row['NEXT_PAGE']: float(row['TRANSITION_PROBABILITY']) for row in result}
    return transitions


def predict_next_page(session, current_page, device_type, time_of_day, params):
    """
    Mode: predict_next
    Returns the most likely next pages with their probabilities
    """
    top_n = params.get('top_n', 5)
    
    if not current_page:
        return {"error": "current_page is required for predict_next mode"}
    
    transitions = get_transitions(session, current_page, device_type, time_of_day)
    
    if not transitions:
        return {
            "error": f"No transitions found for page '{current_page}' on {device_type} during {time_of_day}",
            "current_page": current_page,
            "device_type": device_type,
            "time_of_day": time_of_day
        }
    
    # Sort by probability and take top N
    sorted_transitions = sorted(transitions.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    return {
        "mode": "predict_next",
        "current_page": current_page,
        "device_type": device_type,
        "time_of_day": time_of_day,
        "predictions": [
            {
                "next_page": page,
                "probability": round(prob, 4),
                "probability_pct": round(prob * 100, 2)
            }
            for page, prob in sorted_transitions
        ],
        "total_options": len(transitions)
    }


def calculate_conversion_probability(session, current_page, device_type, time_of_day, params):
    """
    Mode: conversion_probability
    Calculates probability of eventually reaching checkout from current page
    Uses iterative approach: considers all possible paths
    """
    max_steps = params.get('max_steps', 10)
    
    if not current_page:
        return {"error": "current_page is required for conversion_probability mode"}
    
    if current_page == 'checkout':
        return {
            "mode": "conversion_probability",
            "current_page": current_page,
            "conversion_probability": 1.0,
            "conversion_probability_pct": 100.0,
            "message": "Already at checkout"
        }
    
    if current_page == 'EXIT':
        return {
            "mode": "conversion_probability",
            "current_page": current_page,
            "conversion_probability": 0.0,
            "conversion_probability_pct": 0.0,
            "message": "Already exited"
        }
    
    # Track probability of being at each page after N steps
    # current_state[page] = probability of being at that page
    current_state = {current_page: 1.0}
    conversion_prob = 0.0
    
    for step in range(max_steps):
        next_state = defaultdict(float)
        
        for page, prob in current_state.items():
            if page in ['checkout', 'EXIT']:
                # Absorbing states
                if page == 'checkout':
                    conversion_prob += prob
                continue
            
            # Get transitions from this page
            transitions = get_transitions(session, page, device_type, time_of_day)
            
            if not transitions:
                # Dead end - treat as exit
                continue
            
            # Distribute probability to next pages
            for next_page, trans_prob in transitions.items():
                if next_page == 'checkout':
                    conversion_prob += prob * trans_prob
                elif next_page != 'EXIT':
                    next_state[next_page] += prob * trans_prob
        
        current_state = dict(next_state)
        
        # If all probability is absorbed (checkout or exit), stop
        if sum(current_state.values()) < 0.001:
            break
    
    return {
        "mode": "conversion_probability",
        "current_page": current_page,
        "device_type": device_type,
        "time_of_day": time_of_day,
        "conversion_probability": round(conversion_prob, 4),
        "conversion_probability_pct": round(conversion_prob * 100, 2),
        "steps_simulated": step + 1,
        "remaining_probability": round(sum(current_state.values()), 4)
    }


def simulate_journey(session, current_page, device_type, time_of_day, params):
    """
    Mode: simulate_journey
    Runs Monte Carlo simulation of customer journeys
    """
    num_simulations = params.get('num_simulations', 1000)
    max_steps = params.get('max_steps', 20)
    
    if not current_page:
        return {"error": "current_page is required for simulate_journey mode"}
    
    outcomes = []
    path_lengths = []
    all_pages_visited = Counter()
    
    for _ in range(num_simulations):
        page = current_page
        steps = 0
        path = [page]
        
        while steps < max_steps and page not in ['checkout', 'EXIT']:
            transitions = get_transitions(session, page, device_type, time_of_day)
            
            if not transitions:
                # Dead end - treat as exit
                page = 'EXIT'
                break
            
            # Choose next page based on probabilities
            pages = list(transitions.keys())
            probs = list(transitions.values())
            page = random.choices(pages, weights=probs, k=1)[0]
            
            path.append(page)
            steps += 1
        
        outcomes.append(page)
        path_lengths.append(len(path))
        
        # Count page visits
        for p in path:
            all_pages_visited[p] += 1
    
    # Calculate statistics
    outcome_counts = Counter(outcomes)
    conversions = outcome_counts.get('checkout', 0)
    exits = outcome_counts.get('EXIT', 0)
    
    return {
        "mode": "simulate_journey",
        "starting_page": current_page,
        "device_type": device_type,
        "time_of_day": time_of_day,
        "num_simulations": num_simulations,
        "outcomes": {
            "converted": conversions,
            "exited": exits,
            "incomplete": num_simulations - conversions - exits
        },
        "conversion_rate": round(conversions / num_simulations, 4),
        "conversion_rate_pct": round(conversions / num_simulations * 100, 2),
        "exit_rate": round(exits / num_simulations, 4),
        "exit_rate_pct": round(exits / num_simulations * 100, 2),
        "avg_path_length": round(sum(path_lengths) / len(path_lengths), 2),
        "min_path_length": min(path_lengths),
        "max_path_length": max(path_lengths),
        "most_visited_pages": [
            {"page": page, "visits": count, "avg_per_journey": round(count / num_simulations, 2)}
            for page, count in all_pages_visited.most_common(10)
        ]
    }


def find_common_paths(session, device_type, time_of_day, params):
    """
    Mode: common_paths
    Finds most common paths between start and end pages
    Uses actual transition data
    """
    start_page = params.get('start_page', 'home')
    end_page = params.get('end_page', 'checkout')
    top_n = params.get('top_n', 5)
    max_length = params.get('max_length', 6)
    
    # Get all transitions for this device/time combination
    query = f"""
        SELECT CURRENT_PAGE, NEXT_PAGE, TRANSITION_PROBABILITY
        FROM TRANSITION_MATRIX
        WHERE DEVICE_TYPE = '{device_type}'
          AND TIME_OF_DAY = '{time_of_day}'
    """
    
    result = session.sql(query).collect()
    
    # Build adjacency list
    transitions = defaultdict(list)
    for row in result:
        transitions[row['CURRENT_PAGE']].append({
            'page': row['NEXT_PAGE'],
            'prob': float(row['TRANSITION_PROBABILITY'])
        })
    
    # Find paths using DFS with probability tracking
    def find_paths_dfs(current, end, path, probability, max_len):
        if len(path) > max_len:
            return []
        
        if current == end:
            return [(list(path), probability)]
        
        if current not in transitions or current == 'EXIT':
            return []
        
        all_paths = []
        for next_trans in transitions[current]:
            next_page = next_trans['page']
            if next_page not in path:  # Avoid cycles
                path.append(next_page)
                paths = find_paths_dfs(
                    next_page, 
                    end, 
                    path, 
                    probability * next_trans['prob'],
                    max_len
                )
                all_paths.extend(paths)
                path.pop()
        
        return all_paths
    
    # Find all paths
    all_paths = find_paths_dfs(start_page, end_page, [start_page], 1.0, max_length)
    
    if not all_paths:
        return {
            "mode": "common_paths",
            "start_page": start_page,
            "end_page": end_page,
            "device_type": device_type,
            "time_of_day": time_of_day,
            "message": f"No paths found between {start_page} and {end_page} within {max_length} steps",
            "paths": []
        }
    
    # Sort by probability
    all_paths.sort(key=lambda x: x[1], reverse=True)
    top_paths = all_paths[:top_n]
    
    return {
        "mode": "common_paths",
        "start_page": start_page,
        "end_page": end_page,
        "device_type": device_type,
        "time_of_day": time_of_day,
        "total_paths_found": len(all_paths),
        "paths": [
            {
                "path": " → ".join(path),
                "steps": len(path),
                "probability": round(prob, 6),
                "probability_pct": round(prob * 100, 4)
            }
            for path, prob in top_paths
        ]
    }

$$;



### Example Usage

In [ ]:
-- Example 1: Predict next page from cart on mobile in afternoon
CALL MARKOV_CHAIN_ANALYSIS(
    'predict_next',
    'cart',
    'mobile',
    'afternoon',
    '{"top_n": 5}'
);


In [ ]:
-- Example 2: Calculate conversion probability from product_detail
CALL MARKOV_CHAIN_ANALYSIS(
    'conversion_probability',
    'product_detail',
    'desktop',
    'evening',
    '{"max_steps": 10}'
);


In [ ]:
-- Example 3: Simulate 1000 journeys starting from home
CALL MARKOV_CHAIN_ANALYSIS(
    'simulate_journey',
    'home',
    'mobile',
    'afternoon',
    '{"num_simulations": 1000, "max_steps": 20}'
);

In [ ]:
-- Example 4: Find common paths from home to checkout
CALL MARKOV_CHAIN_ANALYSIS(
    'common_paths',
    NULL,
    'desktop',
    'morning',
    '{"start_page": "home", "end_page": "checkout", "top_n": 5, "max_length": 6}'
);
